In [1]:
import os
import cv2
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from collections import Counter

from torch.utils.data import DataLoader
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
# from torchvision.transforms import functional as F
from torch.nn import functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import torchvision.models as models
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# Own
from waste_dataset import WasteDatasetImages, SUPERCATEGORIES
from classifier.resnet18 import resnet18#, resnet18_inference # Error?
from R_CNN import* 
from utility import*

The code will run on CPU.


In [2]:
plot_images_jupyter

# def plot_image_with_boxes(image, bboxes, labels, title, filename):
#     fig, ax = plt.subplots(1)
#     ax.imshow(image)
#     colors = ['r', 'g', 'b', 'y', 'm', 'c', 'k', 'w']
    
#     for i, (bbox, label) in enumerate(zip(bboxes, labels)):
#         x1, y1, x2, y2 = bbox
#         width = x2 - x1
#         height = y2 - y1
#         rect = patches.Rectangle((x1, y1), width, height, linewidth=1, edgecolor=colors[i % len(colors)], facecolor='none')
#         ax.add_patch(rect)
#         ax.text(x1, y1, str(label), color=colors[i % len(colors)])

#     plt.title(title)
#     plt.savefig(filename)
#     plt.close()


<function R_CNN.plot_images_jupyter(images_og, image_idx, train_proposals, train_label, num_images=8)>

In [2]:
patch_size = (64,64)
batch_size = 32

dataset = WasteDatasetImages(transform=transforms.ToTensor(), resize=(256,256))
num_classes = dataset.num_categories()
print(f"Total number of categories: {num_classes}")

# Split the dataset into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

max_proposals_per_image = 1000 # Selective search will generate max 1000 proposals per image
num_images_to_process_train = int(len(train_dataset)/2) #Amount of train images to process
num_images_to_process_test = 20 #Amount of test images to process

# Create dataloaders for train and test sets
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_dataloader  = DataLoader(test_dataset,  batch_size=1, shuffle=False)

# Run selective search 
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

print("Inference")

# model = resnet18_inference(8)  # Replace with your actual model class
model = resnet18(8)  # Replace with your actual model class
# model.load_state_dict(torch.load("trained_models/resnet18_model_10.pth"))
model.eval()

#train_data, train_proposals, train_proposals_image, train_label, images_og, image_idx
_, proposals_box_list, resized_images, _, _, image_idx = generate_proposals_and_labels(test_dataloader, ss, num_images_to_process_test, max_proposals_per_image, img_shape=patch_size)



Total number of categories: 8
Inference


Processing images: 100%|██████████| 20/20 [00:13<00:00,  1.45it/s]
/zhome/a7/d/156506/myfolder/DLCV_GRP_23/Projects/Project4/src/R_CNN.py:225: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  images = torch.tensor(images)


In [3]:
resized_images_array = np.array(resized_images, dtype=np.float32)
resized_images_tensor = torch.from_numpy(resized_images_array)

raw_scores = model(resized_images_tensor)

# Convert raw scores to probabilities
probabilities = F.softmax(raw_scores, dim=1)
#print("probabilities", probabilities)

predicted_labels = torch.argmax(probabilities, dim=1)

bbox_label_pairs = []
for i, (bbox, label) in enumerate(zip(proposals_box_list, predicted_labels.tolist())):
    if label != 0:
        probability = probabilities[i][label].item()
        bbox_label_pairs.append([bbox, label, probability])
# probabilities[0][7].item()

In [6]:
# predicted_bbox_label_vis_per_image = []  # Store the predicted bounding boxes and labels per image
# current_index = 0

# for img_bboxes in proposals_per_image:
#     img_predictions = []
#     for bbox in img_bboxes:
#         label = predicted_labels[current_index].item()
#         if label != 0:
#             probability = probabilities[current_index][label].item()
#             img_predictions.append([bbox, label])
#         current_index += 1
#     predicted_bbox_label_vis_per_image.append(img_predictions)

In [33]:
ctest = 0
ctrain = 0
real_bbox_label_pairs = []      # list[list[float], int]
real_bbox_label_image_vis = []  # for visualization purposes

predicted_bbox_label_vis = []   # for visualization purposes

for batch in test_dataloader:
    if ctest >= num_images_to_process_test:
        break
    images, bboxes, labels = batch
    for idx in range(len(bboxes)):
        bboxes[idx] = [coord.item() for coord in bboxes[idx]]  # Convert tensor to scalar
        real_bbox_label_pairs.append([bboxes[idx], labels[idx].item()])
        
    real_bbox_label_image_vis.append({
        "image": images[0].numpy(),  # Convert tensor to numpy array
         "bbox": bboxes,
        "label": [label.item for label in labels]
    })
            
    ctest += 1

In [20]:
def plot_image_with_boxes(dict_info, name):
    colors = ['r', 'g', 'b', 'y', 'm', 'c', 'k', 'w']
    fig, ax = plt.subplots()
    ax.imshow(dict_info["image"].transpose(1, 2, 0))
    
    for i, (bbox, label) in enumerate(zip(dict_info["bbox"], dict_info["label"])):
        x1, y1, x2, y2 = bbox
        width = x2 - x1
        height = y2 - y1
        rect = patches.Rectangle((x1, y1), width, height, linewidth=1, edgecolor=colors[i % len(colors)], facecolor='none')
        ax.add_patch(rect)
        ax.text(x1, y1, str(label), color=colors[i % len(colors)])
    plt.title(f"{name}_img")
    plt.savefig(f"{name}_img.png")

In [31]:

# print("Real bounding box and label pairs for visualization: ", real_bbox_label_image_vis)
# print("Predicted bounding box and label pairs for visualization: ", predicted_bbox_label_vis_per_image)
# print("Predicted bounding box and label pairs for visualization: ", None)

print("bbox before no max", bbox_label_pairs)
bbox_new = no_max_supression(bbox_label_pairs, 0.5, 0.4)
print("bbox after no max", bbox_new)
assert len(bbox_new) != 0, "Error: bbox is empty and current code can't handle it"

average_precision, precision, recall = mean_average_precision(real_bbox_label_pairs, bbox_new)
print("average_precision", average_precision)
print("precision", precision)
print("recall", recall)

os.makedirs("plots", exist_ok=True)
for idx in range(8):
    plot_image_with_boxes(real_bbox_label_image_vis[idx], f"GT{idx}")
    
    # plot_image_with_boxes(real_bbox_label_image_vis[idx], "GT")



bbox before no max [[[54, 161, 15, 20], 6, 0.2346305251121521], [[108, 165, 17, 12], 2, 0.22893688082695007], [[108, 165, 18, 13], 2, 0.27918609976768494], [[0, 51, 37, 36], 4, 0.3563360571861267], [[228, 159, 28, 26], 6, 0.26635584235191345], [[110, 183, 49, 38], 4, 0.1736883819103241], [[178, 105, 24, 29], 2, 0.1728486269712448], [[228, 0, 28, 22], 6, 0.21350903809070587], [[190, 121, 12, 13], 6, 0.16983948647975922], [[0, 58, 23, 32], 4, 0.36762821674346924], [[10, 207, 21, 20], 6, 0.16170525550842285], [[101, 84, 22, 10], 6, 0.25360211730003357], [[99, 84, 25, 10], 6, 0.3055010437965393], [[99, 85, 23, 10], 6, 0.4451828896999359], [[99, 85, 24, 10], 6, 0.44041821360588074], [[151, 69, 29, 25], 3, 0.18938574194908142], [[0, 0, 256, 256], 2, 0.22453895211219788], [[126, 176, 26, 13], 2, 0.23466433584690094], [[0, 5, 104, 18], 2, 0.2582848072052002], [[56, 118, 140, 72], 6, 0.16993197798728943], [[122, 88, 23, 12], 3, 0.2591009736061096], [[153, 92, 19, 31], 2, 0.2651243209838867], [[

ZeroDivisionError: division by zero